In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
import heapq
from keybert import KeyBERT
import nltk

In [65]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.summarize = self.get_summarize()
        self.word = self.get_word() 
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_word(self):
        self.word_value = word_tokenize(self.sentence, engine="newmm")
        # Iterate over the keys in the dictionary
        return self.word_value
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [3]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  try:
    body_tag = soup.find('body')
    text_below_body = body_tag.get_text().lower()
  except:
    text_below_body = 'not found'
  body_list =[]
  body_list.append(text_below_body)
  return (body_list,title_tag)

In [135]:
def check_lang(url:str):
    data_lang = scrape_tags(url)
    percent = pythainlp.util.countthai(data_lang[0][0])
    if percent >50:
        thai_nlp = Thai(data_lang[0]) 
        a= thai_nlp.word
        new_list = [s.strip().replace('"', '') for s in a if s.strip()]
        while '' in new_list:
            new_list.remove('')

    else:
        return print("English language")

In [41]:
check_lang('https://www.bbc.com/afrique/media/photogalleries')

English language


In [136]:
check_lang('https://www.thairath.co.th/news/crime/2633816')

['ไทยรัฐ', 'สำนักข่าว', 'อันดับ', '1', 'ของ', 'ไทย', '|', 'ไทยรัฐ', 'ออนไลน์', 'ไทยรัฐ', 'ออนไลน์', 'ข่าว', 'พระ', 'ราชสำนัก', 'ทั่ว', 'ไทย', 'ใน', 'กระแส', 'การเมือง', 'เศรษฐกิจ', 'ต่างประเทศ', 'อาชญากรรม', 'ยานยนต์', 'เทคโนโลยี', 'ราคา', 'ทองคำ', 'ราย', 'งานพิเศษ', 'วิดีโอ', 'หนังสือพิมพ์', 'ไทยรัฐ', 'ทีวี', 'ดู', 'ย้อนหลัง', 'ผัง', 'รายการ', 'live', 'ไลฟ์สไตล์', 'กีฬา', 'ฟุตบอล', 'ต่างประเทศ', 'ฟุตบอล', 'ไทย', 'sport', 'insider', 'ไฟต์', 'สปอร์ต', 'กีฬา', 'โลก', 'วิดีโอ', 'แกลเลอรี่', 'ฟุตบอล', 'โลก', '2022', 'บันเทิง', 'ดวง', 'หวย', 'นิยาย', 'โปรโมชั่น', 'ซื้อ', '-', 'ขาย', 'ส่วนลด', 'เช็ค', 'ราคา', 'thairathplus', 'ยิง', 'ดับ', 'พล.ต.ท.', 'ปัญญา', 'ผบช.น.', 'ยัน', 'ฝีมือ', 'ภรรยา', 'อยู่', 'ใน', 'อาการ', 'ช็อก', 'ยัง', 'ให้การ', 'ไม่', 'ได้', 'livelive', 'เข้าสู่ระบบ', 'สมัคร', 'สมาชิก', 'ค้นหา', 'hamburgerhamburger', 'ค้นหา', 'thairath', 'membership', 'เข้าสู่ระบบ', 'สมัคร', 'สมาชิก', 'live', 'ชม', 'รายการ', 'สด', 'ข่าว', 'back', 'ข่าว', 'ข่าว', 'ล่าสุด', 'พระ', 'ราชสำนัก', 'ทั่ว

In [137]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree

def get_continuous_chunks(text,label):
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree and subtree.label() == label:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        if current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk
def eng_location(doc):
    return get_continuous_chunks(doc, 'GPE')

In [143]:
eng_location('India')

['India']